In [36]:
!pip install transformers

In [37]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [51]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


#2. Get Blog Post from Medium

In [39]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"

In [40]:
r = requests.get(URL)

In [42]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])

In [43]:
results

[<p class="be b dw dx dy dz ea eb ec ed ee ef dt"><span><a class="be b dw dx eg dy dz eh ea eb ei ec ed ej ee ef ek el em eo ep eq er es et eu ev ew ex ey ez fa bl fb" data-testid="headerSignUpButton" href="https://medium.com/m/signin?operation=register&amp;redirect=https%3A%2F%2Ftowardsdatascience.com%2Fa-bayesian-take-on-model-regularization-9356116b6457&amp;source=post_page---two_column_layout_nav-----------------------global_nav-----------" rel="noopener follow">Sign up</a></span></p>,
 <p class="be b dw dx dy dz ea eb ec ed ee ef dt"><span><a class="af ag ah ai aj ak al am an ao ap aq ar as at" data-testid="headerSignInButton" href="https://medium.com/m/signin?operation=login&amp;redirect=https%3A%2F%2Ftowardsdatascience.com%2Fa-bayesian-take-on-model-regularization-9356116b6457&amp;source=post_page---two_column_layout_nav-----------------------global_nav-----------" rel="noopener follow">Sign In</a></span></p>,
 <p class="be b dw dx dy dz ea eb ec ed ee ef dt"><span><a class="be 

In [44]:
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [45]:
ARTICLE

'Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Share I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babies lear

# 3. Chunk Text

In [46]:
ARTICLE = ARTICLE.replace('.','.<eos>') # end-of-sentence tag
ARTICLE = ARTICLE.replace('!','!<eos>')
ARTICLE = ARTICLE.replace('?','?<eos>')
sentences = ARTICLE.split('<eos>')

In [47]:
sentences

['Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Share I’m currently reading “How We Learn” by Stanislas Dehaene.',
 ' First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI.',
 ' One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.',
 'e.',
 ' given only a limited amount of experience¹.',
 ' One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world.',
 ' In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models.',
 ' But why do we do this, even from birth¹?',
 ' One argument is that, contrary to the frequentist view in chil

In [48]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
  if len(chunks) == current_chunk + 1:
    if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk += 1
      chunks.append(sentence.split(' '))
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))

0


In [49]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id]=' '.join(chunks[chunk_id])

# 4. Summarize Text

In [54]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [60]:
res

[{'summary_text': ' A Bayesian take on model regularization from a Bayesian perspective . In machine learning, regularization, or model complexity control, is an essential practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models .'}]

In [57]:
' '.join([summ['summary_text'] for summ in res])

' A Bayesian take on model regularization from a Bayesian perspective . In machine learning, regularization, or model complexity control, is an essential practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models .'

# 5. Output Text File

In [61]:
text = ' '.join([summ['summary_text'] for summ in res])

In [64]:
with open('blogsummary.txt','w') as f:
  f.write(text)